In [38]:
# импортируем библиотеки,
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

fr = pd.read_csv('cian.csv')
fr = fr.set_index('Unnamed: 0')

df = fr.copy()

**ШАГ 0. Пишем функцию для проверки количества пропущенных значений.**

Это необходимо для того, чтобы посмотреть какие знаения в каких столбцах у нас пропущенны.

In [41]:
def display_missing(df):    
    for col in df.columns.tolist():          
        print('{} Количество пропущенных значений: {}'.format(col, df[col].isnull().sum()))
    print('\n')

display_missing(df)

ID  объявления Количество пропущенных значений: 0
Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Телефоны Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Площадь комнат, м2 Количество пропущенных значений: 8910
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Санузел Количество пропущенных значений: 2672
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Название ЖК Количество пропущенных значений: 17520
Серия дома Количество пропущенных значений: 21205
Высота потолков, м Количество пропущенных значений: 12162

Убираем слобцы, которые не особо влияют на прогнозирование модели: `Ссылка на объявление`,  `Телефоны`,  `ID  объявления`,  `Тип`, `Серия дома`

In [45]:
df = df.drop(['ID  объявления', 'Ссылка на объявление', 'Телефоны', 'Тип', 'Серия дома'], axis=1)

**ШАГ 1. Проверяем тип данных каждого столбца, для того, чтобы понять какие данные необходимо будет закодировать**

In [46]:
df.dtypes

Количество комнат            object
Метро                        object
Адрес                        object
Площадь, м2                  object
Дом                          object
Парковка                     object
Цена                         object
Описание                     object
Ремонт                       object
Площадь комнат, м2           object
Балкон                       object
Окна                         object
Санузел                      object
Можно с детьми/животными     object
Дополнительно                object
Название ЖК                  object
Высота потолков, м          float64
Лифт                         object
Мусоропровод                 object
dtype: object

Разбиваем столбец Адрес

In [47]:
adres_tab = df['Адрес'].str.split(',', expand=True)
adres_tab = adres_tab.drop([3, 4, 5, 6, 7], axis=1)
adres_tab.columns = ['City', 'Street', 'Number'] 
final = pd.concat([df, adres_tab], axis=1)

In [48]:
final.dtypes

Количество комнат            object
Метро                        object
Адрес                        object
Площадь, м2                  object
Дом                          object
Парковка                     object
Цена                         object
Описание                     object
Ремонт                       object
Площадь комнат, м2           object
Балкон                       object
Окна                         object
Санузел                      object
Можно с детьми/животными     object
Дополнительно                object
Название ЖК                  object
Высота потолков, м          float64
Лифт                         object
Мусоропровод                 object
City                         object
Street                       object
Number                       object
dtype: object

In [49]:
final.head(5)

,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,"Площадь комнат, м2",...,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод,City,Street,Number
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,...,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",3.0,"Пасс (4), Груз (1)",Да,Москва,улица Новый Арбат,27
1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,...,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,3.5,"Пасс (1), Груз (1)",Нет,Москва,улица Новый Арбат,27
2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,...,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,3.2,Пасс (1),NaN,Москва,улица Новый Арбат,27
3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,...,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,3.2,Пасс (1),NaN,Москва,переулок Плотников,21С1
4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,...,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,3.9,"Пасс (1), Груз (1)",Да,Москва,улица Новый Арбат,15


Используем Dummy-кодирование, для кодирования городов

In [51]:
def code_mode(data, feature):
    for i in data[feature].unique():
        data[feature + '=' + i] = (data[feature] == i).astype(float)

code_mode(final, 'City')   

In [52]:
final.head(10)

,Количество комнат,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,"Площадь комнат, м2",...,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод,City,Street,Number,City=Москва,City=Санкт-Петербург,City=Краснодарский край
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,4,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,NaN,...,"Новый Арбат, 2010",3.0,"Пасс (4), Груз (1)",Да,Москва,улица Новый Арбат,27,1.0,0.0,0.0
1,4,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,25 25 20 25,...,Новый Арбат,3.5,"Пасс (1), Груз (1)",Нет,Москва,улица Новый Арбат,27,1.0,0.0,0.0
2,"4, Оба варианта",м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,NaN,...,Новый Арбат,3.2,Пасс (1),NaN,Москва,улица Новый Арбат,27,1.0,0.0,0.0
3,"4, Оба варианта",м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,14-42-20-19,...,NaN,3.2,Пасс (1),NaN,Москва,переулок Плотников,21С1,1.0,0.0,0.0
4,2,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,20 18,...,The Book,3.9,"Пасс (1), Груз (1)",Да,Москва,улица Новый Арбат,15,1.0,0.0,0.0
5,3,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",ID 743. С коллегами работаем 50/50. Видовые ап...,Дизайнерский,NaN,...,Smolensky De Luxe,NaN,NaN,NaN,Москва,1-й Смоленский переулок,21,1.0,0.0,0.0
6,"5, Оба варианта",м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",213.0/140.0/20.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",ID 41980: Предлагается 5-и комнатная квартира...,Евроремонт,43-25-25-25-22,...,NaN,3.2,Пасс (1),NaN,Москва,переулок Романов,3С1,1.0,0.0,0.0
7,3,м. Смоленская (3 мин пешком),"Москва, улица Арбат, 43С3",98.0/63.0/9.0,"2/4, Монолитный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Лот 112453. Ирина Панченко.\n\nБонус агенту 40...,Евроремонт,26 22 15,...,NaN,3.2,Пасс (1),Нет,Москва,улица Арбат,43С3,1.0,0.0,0.0
8,3,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0/95.0/10.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Лот 71833. Евгений Николаев.\n\nБонус агенту 1...,Евроремонт,45 25 25,...,NaN,3.0,Пасс (1),Нет,Москва,Новинский бульвар,18С1,1.0,0.0,0.0


In [1280]:
final['Street'] == 'улица Новый Арбат'

Unnamed: 0
0        False
1        False
2        False
3        False
4        False
         ...  
23363    False
23364    False
23365    False
23366    False
23367    False
Name: Street, Length: 23368, dtype: bool

In [1281]:
final = final.drop(['Серия дома'], axis=1)

In [1282]:
final[final['Street'].str.contains(r'\bулица Новый Арбат')].head(10)

,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,...,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод,City,Street,Number,City=Москва,City=Санкт-Петербург,City=Краснодарский край
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,...,"Новый Арбат, 2010",3.00,"Пасс (4), Груз (1)",Да,Москва,улица Новый Арбат,27,1.0,0.0,0.0
1,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,...,Новый Арбат,3.50,"Пасс (1), Груз (1)",Нет,Москва,улица Новый Арбат,27,1.0,0.0,0.0
2,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,...,Новый Арбат,3.20,Пасс (1),NaN,Москва,улица Новый Арбат,27,1.0,0.0,0.0
4,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,...,The Book,3.90,"Пасс (1), Груз (1)",Да,Москва,улица Новый Арбат,15,1.0,0.0,0.0
16,2,Квартира,м. Арбатская (10 мин пешком),"Москва, улица Новый Арбат, 15",30.0/20.0,19/26,NaN,"120000.0 руб./ За месяц, Залог - 120000 руб., ...",Лот: 86483. БОНУС! Предлагаются эксклюзивные...,Евроремонт,...,The Book,NaN,NaN,NaN,Москва,улица Новый Арбат,15,1.0,0.0,0.0
20,"2, Оба варианта",Квартира,м. Арбатская (9 мин пешком),"Москва, улица Новый Арбат, 15",64.0/50.0/8.0,12/26,NaN,"205000.0 руб./ За месяц, Залог - 205000 руб., ...",В Аренду предлагается просторный апартамент с ...,Евроремонт,...,"The Book, 2020",NaN,NaN,Нет,Москва,улица Новый Арбат,15,1.0,0.0,0.0
28,3,Квартира,м. Арбатская (15 мин пешком),"Москва, улица Новый Арбат, 32",132.8/106.0,"5/13, Монолитный",NaN,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Лот: 82506. Екатерина. БОНУС! Предлагаются э...,Евроремонт,...,Звёзды Арбата,NaN,NaN,NaN,Москва,улица Новый Арбат,32,1.0,0.0,0.0
31,3,Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 32",112.0,"3/13, Монолитно-кирпичный",подземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",Кристина Кабанова. С коллегами работаем 50/50!...,NaN,...,Звёзды Арбата,3.15,NaN,Да,Москва,улица Новый Арбат,32,1.0,0.0,0.0
32,2,Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 32",91.0,"10/13, Монолитно-кирпичный",подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Кристина Кабанова. С коллегами работаем 50/50!...,NaN,...,Звёзды Арбата,3.15,NaN,Да,Москва,улица Новый Арбат,32,1.0,0.0,0.0


In [1283]:
final[['Мусоропровод', 'City', 'Street', 'Number']].head(10)

,Мусоропровод,City,Street,Number
Unnamed: 0,,,,
0,Да,Москва,улица Новый Арбат,27
1,Нет,Москва,улица Новый Арбат,27
2,NaN,Москва,улица Новый Арбат,27
3,NaN,Москва,переулок Плотников,21С1
4,Да,Москва,улица Новый Арбат,15
5,NaN,Москва,1-й Смоленский переулок,21
6,NaN,Москва,переулок Романов,3С1
7,Нет,Москва,улица Арбат,43С3
8,Нет,Москва,Новинский бульвар,18С1


In [1284]:
from cmath import nan


# t = pd.DataFrame(
#     {
#      'Мусоропровод': ['Да', 'Нет', nan, 'Да', nan, 'Нет', nan],
#      'City': ['Москва', 'Москва', 'Москва', 'Санкт-Петербург', 'Москва', 'Москва', 'Санкт-Петербург'],
#      'Street': ['улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат'],
#      'Number': [27, 25, 27, 3, 27, 27, 3]
#     })

In [1285]:
# t

In [1286]:
def make(data, c1, c2, c3):
    data['adres_t'] = data[c1].astype(str) + ' ' + data[c2].astype(str) + ' ' + data[c3].astype(str)
    return (data)

In [1287]:
# t = make(t, 'City', 'Street', 'Number')

In [1288]:
# t

In [1289]:
# for index, row in t[t['Мусоропровод'].isnull() == True].iterrows():
#     m_count = t[(t['adres_t'] == row['adres_t']) & (t['Мусоропровод'].isnull() == False)]['Мусоропровод'].value_counts()
#     yes = m_count['Да'] if 'Да' in m_count else 0
#     no = m_count['Нет'] if 'Нет' in m_count else 0
#     if yes > no:
#         t.at[index, 'Мусоропровод'] = 'Да'
#     else:
#         t.at[index, 'Мусоропровод'] = 'Нет'

In [1290]:
# t

In [1291]:
final.head(2)

,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,...,Название ЖК,"Высота потолков, м",Лифт,Мусоропровод,City,Street,Number,City=Москва,City=Санкт-Петербург,City=Краснодарский край
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,...,"Новый Арбат, 2010",3.0,"Пасс (4), Груз (1)",Да,Москва,улица Новый Арбат,27,1.0,0.0,0.0
1,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,...,Новый Арбат,3.5,"Пасс (1), Груз (1)",Нет,Москва,улица Новый Арбат,27,1.0,0.0,0.0


In [1292]:
def make(data, c1, c2, c3):
    data['adres_t'] = data[c1].astype(str) + ' ' + data[c2].astype(str) + ' ' + data[c3].astype(str)
    return (data)

In [1293]:
final = make(final, 'City', 'Street', 'Number')

In [1294]:
final.head(10)

,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,...,"Высота потолков, м",Лифт,Мусоропровод,City,Street,Number,City=Москва,City=Санкт-Петербург,City=Краснодарский край,adres_t
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,...,3.0,"Пасс (4), Груз (1)",Да,Москва,улица Новый Арбат,27,1.0,0.0,0.0,Москва улица Новый Арбат 27
1,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,...,3.5,"Пасс (1), Груз (1)",Нет,Москва,улица Новый Арбат,27,1.0,0.0,0.0,Москва улица Новый Арбат 27
2,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,...,3.2,Пасс (1),NaN,Москва,улица Новый Арбат,27,1.0,0.0,0.0,Москва улица Новый Арбат 27
3,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,...,3.2,Пасс (1),NaN,Москва,переулок Плотников,21С1,1.0,0.0,0.0,Москва переулок Плотников 21С1
4,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,...,3.9,"Пасс (1), Груз (1)",Да,Москва,улица Новый Арбат,15,1.0,0.0,0.0,Москва улица Новый Арбат 15
5,3,Квартира,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",ID 743. С коллегами работаем 50/50. Видовые ап...,Дизайнерский,...,NaN,NaN,NaN,Москва,1-й Смоленский переулок,21,1.0,0.0,0.0,Москва 1-й Смоленский переулок 21
6,"5, Оба варианта",Квартира,м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",213.0/140.0/20.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",ID 41980: Предлагается 5-и комнатная квартира...,Евроремонт,...,3.2,Пасс (1),NaN,Москва,переулок Романов,3С1,1.0,0.0,0.0,Москва переулок Романов 3С1
7,3,Квартира,м. Смоленская (3 мин пешком),"Москва, улица Арбат, 43С3",98.0/63.0/9.0,"2/4, Монолитный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Лот 112453. Ирина Панченко.\n\nБонус агенту 40...,Евроремонт,...,3.2,Пасс (1),Нет,Москва,улица Арбат,43С3,1.0,0.0,0.0,Москва улица Арбат 43С3
8,3,Квартира,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0/95.0/10.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Лот 71833. Евгений Николаев.\n\nБонус агенту 1...,Евроремонт,...,3.0,Пасс (1),Нет,Москва,Новинский бульвар,18С1,1.0,0.0,0.0,Москва Новинский бульвар 18С1


In [1295]:
def display_missing(df):    
    for col in df.columns.tolist():          
        print('{} Количество пропущенных значений: {}'.format(col, df[col].isnull().sum()))
    print('\n')

display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Площадь комнат, м2 Количество пропущенных значений: 8910
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Название ЖК Количество пропущенных значений: 17520
Высота потолков, м Количество пропущенных значений: 12162
Лифт Количество пропущенных значений: 5500
Мусоропровод Количество пропущенных значений: 10522
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Колич

In [1296]:
for index, row in final[final['Мусоропровод'].isnull() == True].iterrows():
    m_count = final[(final['adres_t'] == row['adres_t']) & (final['Мусоропровод'].isnull() == False)]['Мусоропровод'].value_counts()
    yes = m_count['Да'] if 'Да' in m_count else 0
    no = m_count['Нет'] if 'Нет' in m_count else 0
    if yes > no:
        final.at[index, 'Мусоропровод'] = 'Да'
    else:
        final.at[index, 'Мусоропровод'] = 'Нет'

In [1297]:
display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Площадь комнат, м2 Количество пропущенных значений: 8910
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Название ЖК Количество пропущенных значений: 17520
Высота потолков, м Количество пропущенных значений: 12162
Лифт Количество пропущенных значений: 5500
Мусоропровод Количество пропущенных значений: 0
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количеств

In [1298]:
final['Garbage_chute'] = final['Мусоропровод'].transform(lambda x: 1 if x=='Да' else 0)

In [1299]:
final.head(10)

,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,Описание,Ремонт,...,Лифт,Мусоропровод,City,Street,Number,City=Москва,City=Санкт-Петербург,City=Краснодарский край,adres_t,Garbage_chute
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Без комиссии для нанимателя! Бонус коллегам 12...,Дизайнерский,...,"Пасс (4), Груз (1)",Да,Москва,улица Новый Арбат,27,1.0,0.0,0.0,Москва улица Новый Арбат 27,1
1,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Лот 93107. Елена Анисимова.\n\nБонус агенту 50...,Дизайнерский,...,"Пасс (1), Груз (1)",Нет,Москва,улица Новый Арбат,27,1.0,0.0,0.0,Москва улица Новый Арбат 27,0
2,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...","ID 36380: Шикарная 4-х км. квартира в ЖК ""Нов...",Евроремонт,...,Пасс (1),Да,Москва,улица Новый Арбат,27,1.0,0.0,0.0,Москва улица Новый Арбат 27,1
3,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",ID 31618: Эксклюзивное предложение. Современн...,Евроремонт,...,Пасс (1),Да,Москва,переулок Плотников,21С1,1.0,0.0,0.0,Москва переулок Плотников 21С1,1
4,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Лот 111542. Татьяна Лучкина.\n\nБонус агенту 5...,Евроремонт,...,"Пасс (1), Груз (1)",Да,Москва,улица Новый Арбат,15,1.0,0.0,0.0,Москва улица Новый Арбат 15,1
5,3,Квартира,м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.0,3/7,NaN,"470000.0 руб./ За месяц, Залог - 470000 руб., ...",ID 743. С коллегами работаем 50/50. Видовые ап...,Дизайнерский,...,NaN,Да,Москва,1-й Смоленский переулок,21,1.0,0.0,0.0,Москва 1-й Смоленский переулок 21,1
6,"5, Оба варианта",Квартира,м. Арбатская (4 мин пешком),"Москва, переулок Романов, 3С1",213.0/140.0/20.0,4/5,наземная,"350000.0 руб./ За месяц, Залог - 350000 руб., ...",ID 41980: Предлагается 5-и комнатная квартира...,Евроремонт,...,Пасс (1),Нет,Москва,переулок Романов,3С1,1.0,0.0,0.0,Москва переулок Романов 3С1,0
7,3,Квартира,м. Смоленская (3 мин пешком),"Москва, улица Арбат, 43С3",98.0/63.0/9.0,"2/4, Монолитный",подземная,"250000.0 руб./ За месяц, Залог - 250000 руб., ...",Лот 112453. Ирина Панченко.\n\nБонус агенту 40...,Евроремонт,...,Пасс (1),Нет,Москва,улица Арбат,43С3,1.0,0.0,0.0,Москва улица Арбат 43С3,0
8,3,Квартира,м. Смоленская (9 мин пешком),"Москва, Новинский бульвар, 18С1",120.0/95.0/10.0,"5/10, Сталинский",открытая,"130000.0 руб./ За месяц, Залог - 130000 руб., ...",Лот 71833. Евгений Николаев.\n\nБонус агенту 1...,Евроремонт,...,Пасс (1),Нет,Москва,Новинский бульвар,18С1,1.0,0.0,0.0,Москва Новинский бульвар 18С1,0


In [1300]:
display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Площадь комнат, м2 Количество пропущенных значений: 8910
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Название ЖК Количество пропущенных значений: 17520
Высота потолков, м Количество пропущенных значений: 12162
Лифт Количество пропущенных значений: 5500
Мусоропровод Количество пропущенных значений: 0
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количеств

In [1301]:
final['Лифт'].head(10)

Unnamed: 0
0    Пасс (4), Груз (1)
1    Пасс (1), Груз (1)
2              Пасс (1)
3              Пасс (1)
4    Пасс (1), Груз (1)
5                   NaN
6              Пасс (1)
7              Пасс (1)
8              Пасс (1)
9              Пасс (1)
Name: Лифт, dtype: object

In [1302]:
final['Дом'].head(10)

Unnamed: 0
0             5/16, Монолитный
1    5/16, Монолитно-кирпичный
2                         5/16
3                          5/6
4             12/26, Панельный
5                          3/7
6                          4/5
7              2/4, Монолитный
8             5/10, Сталинский
9              2/7, Сталинский
Name: Дом, dtype: object

In [1303]:
house_tab = final['Дом'].str.split(', ', expand=True)

In [1304]:
house_tab.columns = ['Levels', 'Type']
house_tab

,Levels,Type
Unnamed: 0,,
0,5/16,Монолитный
1,5/16,Монолитно-кирпичный
2,5/16,None
3,5/6,None
4,12/26,Панельный
...,...,...
23363,10/14,None
23364,5/18,Монолитный
23365,5/5,Кирпичный


In [1305]:
final = pd.concat([final, house_tab], axis=1)

In [1306]:
final = final.drop(['Мусоропровод'], axis=1)

In [1307]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Площадь комнат, м2',
       'Балкон', 'Окна', 'Можно с детьми/животными', 'Дополнительно',
       'Название ЖК', 'Высота потолков, м', 'Лифт', 'City', 'Street', 'Number',
       'City=Москва', 'City=Санкт-Петербург', 'City=Краснодарский край',
       'adres_t', 'Garbage_chute', 'Levels', 'Type'],
      dtype='object')

In [1308]:
numbers = final['Levels'].str.split('/', expand=True)

In [1309]:
numbers.columns = ['Level', 'Count_levels']

In [1310]:
final = pd.concat([final, numbers], axis=1)

In [1311]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Площадь комнат, м2',
       'Балкон', 'Окна', 'Можно с детьми/животными', 'Дополнительно',
       'Название ЖК', 'Высота потолков, м', 'Лифт', 'City', 'Street', 'Number',
       'City=Москва', 'City=Санкт-Петербург', 'City=Краснодарский край',
       'adres_t', 'Garbage_chute', 'Levels', 'Type', 'Level', 'Count_levels'],
      dtype='object')

In [1312]:
final['Type'].unique()

array(['Монолитный', 'Монолитно-кирпичный', None, 'Панельный',
       'Сталинский', 'Кирпичный', 'старый фонд', 'Блочный', 'Деревянный',
       'Газобетонный блок', 'Щитовой'], dtype=object)

In [1313]:
final['Высота потолков, м'].head(10)

Unnamed: 0
0      3.0
1      3.5
2      3.2
3      3.2
4      3.9
5      NaN
6      3.2
7      3.2
8      3.0
9    310.0
Name: Высота потолков, м, dtype: float64

In [1314]:
k = pd.DataFrame(
    {
    'Мусоропровод': ['Да', 'Нет', nan, 'Да', nan, 'Нет', nan, 'Нет'],
    'City': ['Москва', 'Москва', 'Москва', 'Санкт-Петербург', 'Москва', 'Москва', 'Санкт-Петербург', 'Москва'],
    'Street': ['улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат'],
    'Number': [27, 25, 27, 3, 27, 27, 3, 4],
    'Высота': [3.2, 3.5, nan, 3.4, 2.5, 300.1, nan, nan]
    }
)

In [1315]:
k

,Мусоропровод,City,Street,Number,Высота
0,Да,Москва,улица Новый Арбат,27,3.2
1,Нет,Москва,улица Новый Арбат,25,3.5
2,NaN,Москва,улица Новый Арбат,27,NaN
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4
4,NaN,Москва,улица Новый Арбат,27,2.5
5,Нет,Москва,улица Новый Арбат,27,300.1
6,NaN,Санкт-Петербург,улица Новый Арбат,3,NaN
7,Нет,Москва,улица Новый Арбат,4,NaN


In [1316]:
k = make(k, 'City', 'Street', 'Number')

In [1317]:
k

,Мусоропровод,City,Street,Number,Высота,adres_t
0,Да,Москва,улица Новый Арбат,27,3.2,Москва улица Новый Арбат 27
1,Нет,Москва,улица Новый Арбат,25,3.5,Москва улица Новый Арбат 25
2,NaN,Москва,улица Новый Арбат,27,NaN,Москва улица Новый Арбат 27
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Санкт-Петербург улица Новый Арбат 3
4,NaN,Москва,улица Новый Арбат,27,2.5,Москва улица Новый Арбат 27
5,Нет,Москва,улица Новый Арбат,27,300.1,Москва улица Новый Арбат 27
6,NaN,Санкт-Петербург,улица Новый Арбат,3,NaN,Санкт-Петербург улица Новый Арбат 3
7,Нет,Москва,улица Новый Арбат,4,NaN,Москва улица Новый Арбат 4


In [1318]:
for index, row in k[k['Мусоропровод'].isnull() == True].iterrows():
    m_count = k[(k['adres_t'] == row['adres_t']) & (k['Мусоропровод'].isnull() == False)]['Мусоропровод'].value_counts()
    yes = m_count['Да'] if 'Да' in m_count else 0
    no = m_count['Нет'] if 'Нет' in m_count else 0
    if yes > no:
        k.at[index, 'Мусоропровод'] = 'Да'
    else:
        k.at[index, 'Мусоропровод'] = 'Нет'

In [1319]:
k

,Мусоропровод,City,Street,Number,Высота,adres_t
0,Да,Москва,улица Новый Арбат,27,3.2,Москва улица Новый Арбат 27
1,Нет,Москва,улица Новый Арбат,25,3.5,Москва улица Новый Арбат 25
2,Нет,Москва,улица Новый Арбат,27,NaN,Москва улица Новый Арбат 27
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Санкт-Петербург улица Новый Арбат 3
4,Нет,Москва,улица Новый Арбат,27,2.5,Москва улица Новый Арбат 27
5,Нет,Москва,улица Новый Арбат,27,300.1,Москва улица Новый Арбат 27
6,Да,Санкт-Петербург,улица Новый Арбат,3,NaN,Санкт-Петербург улица Новый Арбат 3
7,Нет,Москва,улица Новый Арбат,4,NaN,Москва улица Новый Арбат 4


In [1320]:
k['Высота'].describe()

count      5.000000
mean      62.540000
std      132.800651
min        2.500000
25%        3.200000
50%        3.400000
75%        3.500000
max      300.100000
Name: Высота, dtype: float64

In [1321]:
k['Высота'] = np.where((k['Высота'] > 5), 3, k['Высота'])

In [1322]:
k

,Мусоропровод,City,Street,Number,Высота,adres_t
0,Да,Москва,улица Новый Арбат,27,3.2,Москва улица Новый Арбат 27
1,Нет,Москва,улица Новый Арбат,25,3.5,Москва улица Новый Арбат 25
2,Нет,Москва,улица Новый Арбат,27,NaN,Москва улица Новый Арбат 27
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Санкт-Петербург улица Новый Арбат 3
4,Нет,Москва,улица Новый Арбат,27,2.5,Москва улица Новый Арбат 27
5,Нет,Москва,улица Новый Арбат,27,3.0,Москва улица Новый Арбат 27
6,Да,Санкт-Петербург,улица Новый Арбат,3,NaN,Санкт-Петербург улица Новый Арбат 3
7,Нет,Москва,улица Новый Арбат,4,NaN,Москва улица Новый Арбат 4


In [1323]:
for index, row in k[k['Высота'].isnull() == True].iterrows():
    m_count = k[(k['adres_t'] == row['adres_t']) & (k['Высота'].isnull() == False)]['Высота'].mean()
    k.at[index, 'Высота'] = m_count

In [1324]:
k

,Мусоропровод,City,Street,Number,Высота,adres_t
0,Да,Москва,улица Новый Арбат,27,3.2,Москва улица Новый Арбат 27
1,Нет,Москва,улица Новый Арбат,25,3.5,Москва улица Новый Арбат 25
2,Нет,Москва,улица Новый Арбат,27,2.9,Москва улица Новый Арбат 27
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Санкт-Петербург улица Новый Арбат 3
4,Нет,Москва,улица Новый Арбат,27,2.5,Москва улица Новый Арбат 27
5,Нет,Москва,улица Новый Арбат,27,3.0,Москва улица Новый Арбат 27
6,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Санкт-Петербург улица Новый Арбат 3
7,Нет,Москва,улица Новый Арбат,4,NaN,Москва улица Новый Арбат 4


In [1325]:
for index, row in k[k['Высота'].isnull() == True].iterrows():
    m_count = k[(k['City'] == row['City']) & (k['Высота'].isnull() == False)]['Высота'].mean()
    k.at[index, 'Высота'] = m_count

In [1326]:
k

,Мусоропровод,City,Street,Number,Высота,adres_t
0,Да,Москва,улица Новый Арбат,27,3.20,Москва улица Новый Арбат 27
1,Нет,Москва,улица Новый Арбат,25,3.50,Москва улица Новый Арбат 25
2,Нет,Москва,улица Новый Арбат,27,2.90,Москва улица Новый Арбат 27
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.40,Санкт-Петербург улица Новый Арбат 3
4,Нет,Москва,улица Новый Арбат,27,2.50,Москва улица Новый Арбат 27
5,Нет,Москва,улица Новый Арбат,27,3.00,Москва улица Новый Арбат 27
6,Да,Санкт-Петербург,улица Новый Арбат,3,3.40,Санкт-Петербург улица Новый Арбат 3
7,Нет,Москва,улица Новый Арбат,4,3.02,Москва улица Новый Арбат 4


In [1327]:
display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Площадь комнат, м2 Количество пропущенных значений: 8910
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Название ЖК Количество пропущенных значений: 17520
Высота потолков, м Количество пропущенных значений: 12162
Лифт Количество пропущенных значений: 5500
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количество пропущенных значений: 4
City=Москва Количество

In [1328]:
final['Высота потолков, м'] = np.where((final['Высота потолков, м'] > 5), 3, final['Высота потолков, м'])

In [1329]:
for index, row in final[final['Высота потолков, м'].isnull() == True].iterrows():
    f_count = final[(final['adres_t'] == row['adres_t']) & (final['Высота потолков, м'].isnull() == False)]['Высота потолков, м'].mean()
    final.at[index, 'Высота потолков, м'] = f_count

In [1330]:
display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Площадь комнат, м2 Количество пропущенных значений: 8910
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Название ЖК Количество пропущенных значений: 17520
Высота потолков, м Количество пропущенных значений: 6670
Лифт Количество пропущенных значений: 5500
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количество пропущенных значений: 4
City=Москва Количество 

In [1331]:
for index, row in final[final['Высота потолков, м'].isnull() == True].iterrows():
    n_count = final[(final['City'] == row['City']) & (final['Высота потолков, м'].isnull() == False)]['Высота потолков, м'].mean()
    final.at[index, 'Высота потолков, м'] = n_count

In [1332]:
display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Площадь комнат, м2 Количество пропущенных значений: 8910
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Название ЖК Количество пропущенных значений: 17520
Высота потолков, м Количество пропущенных значений: 0
Лифт Количество пропущенных значений: 5500
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количество пропущенных значений: 4
City=Москва Количество про

In [1333]:
final = final.drop(['Площадь комнат, м2'], axis=1)

In [1334]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Название ЖК',
       'Высота потолков, м', 'Лифт', 'City', 'Street', 'Number', 'City=Москва',
       'City=Санкт-Петербург', 'City=Краснодарский край', 'adres_t',
       'Garbage_chute', 'Levels', 'Type', 'Level', 'Count_levels'],
      dtype='object')

In [1335]:
final.dtypes

Количество комнат            object
Тип                          object
Метро                        object
Адрес                        object
Площадь, м2                  object
Дом                          object
Парковка                     object
Цена                         object
Описание                     object
Ремонт                       object
Балкон                       object
Окна                         object
Можно с детьми/животными     object
Дополнительно                object
Название ЖК                  object
Высота потолков, м          float64
Лифт                         object
City                         object
Street                       object
Number                       object
City=Москва                 float64
City=Санкт-Петербург        float64
City=Краснодарский край     float64
adres_t                      object
Garbage_chute                 int64
Levels                       object
Type                         object
Level                       

In [1336]:
final['Лифт'].head(7)

Unnamed: 0
0    Пасс (4), Груз (1)
1    Пасс (1), Груз (1)
2              Пасс (1)
3              Пасс (1)
4    Пасс (1), Груз (1)
5                   NaN
6              Пасс (1)
Name: Лифт, dtype: object

In [1337]:
m = pd.DataFrame(
    {
    'Мусоропровод': ['Да', 'Нет', nan, 'Да', nan, 'Нет', nan, 'Нет'],
    'City': ['Москва', 'Москва', 'Москва', 'Санкт-Петербург', 'Москва', 'Москва', 'Санкт-Петербург', 'Москва'],
    'Street': ['улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат', 'улица Новый Арбат'],
    'Number': [27, 25, 27, 3, 27, 27, 3, 4],
    'Высота': [3.2, 3.5, nan, 3.4, 2.5, 3.1, nan, nan],
    'Лифт': ['Пасс (4), Груз (1)', 'Пасс (1), Груз (1)', 'Груз (1)', 'Пасс (1)', 'Пасс (1), Груз (1)', 'Пасс (2), Груз (1)', 'Пасс (1), Груз (1)', 'Пасс (1), Груз (1)']
    }
)

In [1338]:
m

,Мусоропровод,City,Street,Number,Высота,Лифт
0,Да,Москва,улица Новый Арбат,27,3.2,"Пасс (4), Груз (1)"
1,Нет,Москва,улица Новый Арбат,25,3.5,"Пасс (1), Груз (1)"
2,NaN,Москва,улица Новый Арбат,27,NaN,Груз (1)
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Пасс (1)
4,NaN,Москва,улица Новый Арбат,27,2.5,"Пасс (1), Груз (1)"
5,Нет,Москва,улица Новый Арбат,27,3.1,"Пасс (2), Груз (1)"
6,NaN,Санкт-Петербург,улица Новый Арбат,3,NaN,"Пасс (1), Груз (1)"
7,Нет,Москва,улица Новый Арбат,4,NaN,"Пасс (1), Груз (1)"


In [1339]:
def transf(data, col):
    for i in range (0, 100):
        data[col] = data[col].str.replace(f'({i})', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '')


In [1340]:
transf(m, 'Лифт')

/tmp/ipykernel_22165/1931943582.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(f'({i})', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '')
/tmp/ipykernel_22165/1931943582.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[col] = data[col].str.replace(f'({i})', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '')


In [1341]:
m

,Мусоропровод,City,Street,Number,Высота,Лифт
0,Да,Москва,улица Новый Арбат,27,3.2,ПассГруз
1,Нет,Москва,улица Новый Арбат,25,3.5,ПассГруз
2,NaN,Москва,улица Новый Арбат,27,NaN,Груз
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Пасс
4,NaN,Москва,улица Новый Арбат,27,2.5,ПассГруз
5,Нет,Москва,улица Новый Арбат,27,3.1,ПассГруз
6,NaN,Санкт-Петербург,улица Новый Арбат,3,NaN,ПассГруз
7,Нет,Москва,улица Новый Арбат,4,NaN,ПассГруз


In [1342]:
def code_transform(data, feature):
    for i in data[feature].unique():
        data[feature + '=' + i] = (data[feature] == i).astype(float)

In [1343]:
code_transform(m, 'Лифт')

In [1344]:
m

,Мусоропровод,City,Street,Number,Высота,Лифт,Лифт=ПассГруз,Лифт=Груз,Лифт=Пасс
0,Да,Москва,улица Новый Арбат,27,3.2,ПассГруз,1.0,0.0,0.0
1,Нет,Москва,улица Новый Арбат,25,3.5,ПассГруз,1.0,0.0,0.0
2,NaN,Москва,улица Новый Арбат,27,NaN,Груз,0.0,1.0,0.0
3,Да,Санкт-Петербург,улица Новый Арбат,3,3.4,Пасс,0.0,0.0,1.0
4,NaN,Москва,улица Новый Арбат,27,2.5,ПассГруз,1.0,0.0,0.0
5,Нет,Москва,улица Новый Арбат,27,3.1,ПассГруз,1.0,0.0,0.0
6,NaN,Санкт-Петербург,улица Новый Арбат,3,NaN,ПассГруз,1.0,0.0,0.0
7,Нет,Москва,улица Новый Арбат,4,NaN,ПассГруз,1.0,0.0,0.0


In [1345]:
final['Лифт'].unique()

array(['Пасс (4), Груз (1)', 'Пасс (1), Груз (1)', 'Пасс (1)', nan,
       'Пасс (3)', 'Пасс (4)', 'Пасс (2), Груз (1)', 'Пасс (2)',
       'Пасс (2), Груз (2)', 'Пасс (3), Груз (3)', 'Груз (4)',
       'Пасс (3), Груз (1)', 'Груз (1)', 'Пасс (1), Груз (2)', 'Груз (2)',
       'Пасс (3), Груз (2)', 'Пасс (4), Груз (4)', 'Пасс (2), Груз (3)',
       'Пасс (4), Груз (2)', 'Груз (3)', 'Пасс (1), Груз (3)',
       'Пасс (4), Груз (3)', 'Пасс (5), Груз (1)', 'Пасс (1), Груз (12)',
       'Пасс (60)', 'Пасс (6)', 'Пасс (8), Груз (8)', 'Пасс (7)',
       'Пасс (1), Груз (4)', 'Пасс (50)', 'Пасс (5), Груз (3)',
       'Пасс (2), Груз (4)', 'Груз (6)', 'Груз (8)', 'Пасс (5)'],
      dtype=object)

In [1346]:
transf(final, 'Лифт')

/tmp/ipykernel_22165/1931943582.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data[col] = data[col].str.replace(f'({i})', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '')
/tmp/ipykernel_22165/1931943582.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[col] = data[col].str.replace(f'({i})', '').str.replace('(', '').str.replace(')', '').str.replace(',', '').str.replace(' ', '')


In [1347]:
final['Лифт'].unique()


array(['ПассГруз', 'Пасс', nan, 'Груз'], dtype=object)

In [1348]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Название ЖК',
       'Высота потолков, м', 'Лифт', 'City', 'Street', 'Number', 'City=Москва',
       'City=Санкт-Петербург', 'City=Краснодарский край', 'adres_t',
       'Garbage_chute', 'Levels', 'Type', 'Level', 'Count_levels'],
      dtype='object')

In [1349]:
final = final.drop(['Название ЖК'], axis=1)

In [1350]:
# удалить все запятые в адресе и создать новый столбец и по нему уже сравнивать
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'City=Москва',
       'City=Санкт-Петербург', 'City=Краснодарский край', 'adres_t',
       'Garbage_chute', 'Levels', 'Type', 'Level', 'Count_levels'],
      dtype='object')

In [1351]:
final['Лифт'].fillna('Пасс', inplace=True)

In [1352]:
display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 2755
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Высота потолков, м Количество пропущенных значений: 0
Лифт Количество пропущенных значений: 0
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количество пропущенных значений: 4
City=Москва Количество пропущенных значений: 0
City=Санкт-Петербург Количество пропущенных значений: 0
City=Краснодарский край Количество

In [1353]:
# code_transform(final, 'Лифт')

In [1354]:
final['Лифт'].unique()

array(['ПассГруз', 'Пасс', 'Груз'], dtype=object)

In [1355]:
code_transform(final, 'Лифт')

In [1356]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'City=Москва',
       'City=Санкт-Петербург', 'City=Краснодарский край', 'adres_t',
       'Garbage_chute', 'Levels', 'Type', 'Level', 'Count_levels',
       'Лифт=ПассГруз', 'Лифт=Пасс', 'Лифт=Груз'],
      dtype='object')

In [1357]:
final['Ремонт'].unique()

array(['Дизайнерский', 'Евроремонт', 'Косметический', nan, 'Без ремонта'],
      dtype=object)

In [1358]:
final['Ремонт'].fillna('Не указан', inplace=True)

In [1359]:
final['Ремонт'].unique()

array(['Дизайнерский', 'Евроремонт', 'Косметический', 'Не указан',
       'Без ремонта'], dtype=object)

In [1360]:
code_transform(final, 'Ремонт')

In [1361]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'City=Москва',
       'City=Санкт-Петербург', 'City=Краснодарский край', 'adres_t',
       'Garbage_chute', 'Levels', 'Type', 'Level', 'Count_levels',
       'Лифт=ПассГруз', 'Лифт=Пасс', 'Лифт=Груз', 'Ремонт=Дизайнерский',
       'Ремонт=Евроремонт', 'Ремонт=Косметический', 'Ремонт=Не указан',
       'Ремонт=Без ремонта'],
      dtype='object')

In [1362]:
display_missing(final)

Количество комнат Количество пропущенных значений: 1041
Тип Количество пропущенных значений: 0
Метро Количество пропущенных значений: 1315
Адрес Количество пропущенных значений: 0
Площадь, м2 Количество пропущенных значений: 0
Дом Количество пропущенных значений: 0
Парковка Количество пропущенных значений: 13417
Цена Количество пропущенных значений: 0
Описание Количество пропущенных значений: 0
Ремонт Количество пропущенных значений: 0
Балкон Количество пропущенных значений: 7978
Окна Количество пропущенных значений: 6613
Можно с детьми/животными Количество пропущенных значений: 6096
Дополнительно Количество пропущенных значений: 357
Высота потолков, м Количество пропущенных значений: 0
Лифт Количество пропущенных значений: 0
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количество пропущенных значений: 4
City=Москва Количество пропущенных значений: 0
City=Санкт-Петербург Количество пропущенных значений: 0
City=Краснодарский край Количество пр

In [1363]:
final.rename(columns = {'City=Москва' : 'city_moscow', 'City=Санкт-Петербург' : 'city_saint_petersburg', 'City=Краснодарский край' : 'city_krasnodar_region'}, inplace = True)


In [1364]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'city_moscow',
       'city_saint_petersburg', 'city_krasnodar_region', 'adres_t',
       'Garbage_chute', 'Levels', 'Type', 'Level', 'Count_levels',
       'Лифт=ПассГруз', 'Лифт=Пасс', 'Лифт=Груз', 'Ремонт=Дизайнерский',
       'Ремонт=Евроремонт', 'Ремонт=Косметический', 'Ремонт=Не указан',
       'Ремонт=Без ремонта'],
      dtype='object')

In [1365]:
final = final.drop(['Levels', 'Type'], axis=1)

In [1366]:
final.dtypes

Количество комнат            object
Тип                          object
Метро                        object
Адрес                        object
Площадь, м2                  object
Дом                          object
Парковка                     object
Цена                         object
Описание                     object
Ремонт                       object
Балкон                       object
Окна                         object
Можно с детьми/животными     object
Дополнительно                object
Высота потолков, м          float64
Лифт                         object
City                         object
Street                       object
Number                       object
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
adres_t                      object
Garbage_chute                 int64
Level                        object
Count_levels                 object
Лифт=ПассГруз               float64
Лифт=Пасс                   

In [1367]:
final.rename(columns = {'Garbage_chute' : 'garbage_chute', 'Лифт=Пасс' : 'elevator_passenger', 'Лифт=Груз' : 'elevator_cargo', 'Лифт=ПассГруз' : 'elevator_passenger_cargo', 'Ремонт=Дизайнерский': 'repair_designer', 'Ремонт=Евроремонт' : 'repair_renovation', 'Ремонт=Косметический' : 'repair_redecorating', 'Ремонт=Не указан' : 'repair_not_specified', 'Ремонт=Без ремонта':'repair_without_repair'}, inplace = True)

In [1368]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'city_moscow',
       'city_saint_petersburg', 'city_krasnodar_region', 'adres_t',
       'garbage_chute', 'Level', 'Count_levels', 'elevator_passenger_cargo',
       'elevator_passenger', 'elevator_cargo', 'repair_designer',
       'repair_renovation', 'repair_redecorating', 'repair_not_specified',
       'repair_without_repair'],
      dtype='object')

In [1369]:
final.rename(columns = {'Level' : 'building_level', 'Count_levels' : 'building_сount_levels'}, inplace = True)

In [1370]:
final.dtypes

Количество комнат            object
Тип                          object
Метро                        object
Адрес                        object
Площадь, м2                  object
Дом                          object
Парковка                     object
Цена                         object
Описание                     object
Ремонт                       object
Балкон                       object
Окна                         object
Можно с детьми/животными     object
Дополнительно                object
Высота потолков, м          float64
Лифт                         object
City                         object
Street                       object
Number                       object
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
adres_t                      object
garbage_chute                 int64
building_level               object
building_сount_levels        object
elevator_passenger_cargo    float64
elevator_passenger          

In [1371]:
final.columns

Index(['Количество комнат', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Дом',
       'Парковка', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'Окна',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'city_moscow',
       'city_saint_petersburg', 'city_krasnodar_region', 'adres_t',
       'garbage_chute', 'building_level', 'building_сount_levels',
       'elevator_passenger_cargo', 'elevator_passenger', 'elevator_cargo',
       'repair_designer', 'repair_renovation', 'repair_redecorating',
       'repair_not_specified', 'repair_without_repair'],
      dtype='object')

In [1372]:
final['building_level'] = final['building_level'].astype(int)
final['building_сount_levels'] = final['building_сount_levels'].astype(int)

In [1373]:
final['building_level']

Unnamed: 0
0         5
1         5
2         5
3         5
4        12
         ..
23363    10
23364     5
23365     5
23366     8
23367     6
Name: building_level, Length: 23368, dtype: int64

In [1374]:
final['building_сount_levels']

Unnamed: 0
0        16
1        16
2        16
3         6
4        26
         ..
23363    14
23364    18
23365     5
23366    23
23367    23
Name: building_сount_levels, Length: 23368, dtype: int64

In [1375]:
final.rename(columns = {'Тип' : 'type_of_room', 'Парковка' : 'parking', 'Окна' : 'window'}, inplace = True)

In [1376]:
final.columns

Index(['Количество комнат', 'type_of_room', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'parking', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'window',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'city_moscow',
       'city_saint_petersburg', 'city_krasnodar_region', 'adres_t',
       'garbage_chute', 'building_level', 'building_сount_levels',
       'elevator_passenger_cargo', 'elevator_passenger', 'elevator_cargo',
       'repair_designer', 'repair_renovation', 'repair_redecorating',
       'repair_not_specified', 'repair_without_repair'],
      dtype='object')

In [1377]:
final['parking'] = final['parking'].replace(to_replace ='открытая', value = 'бесплатная', regex = True)
final['parking'] = final['parking'].replace(to_replace ='наземная' or 'подземная' or 'многоуровневая' or 'на крыше', value = 'платная', regex = True)
final['parking'] = final['parking'].fillna('бесплатная')
final['parking'] = final['parking'].transform(lambda x: 1 if x=='платная' else 0)

In [1378]:
final.columns

Index(['Количество комнат', 'type_of_room', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'parking', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'window',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'city_moscow',
       'city_saint_petersburg', 'city_krasnodar_region', 'adres_t',
       'garbage_chute', 'building_level', 'building_сount_levels',
       'elevator_passenger_cargo', 'elevator_passenger', 'elevator_cargo',
       'repair_designer', 'repair_renovation', 'repair_redecorating',
       'repair_not_specified', 'repair_without_repair'],
      dtype='object')

In [1379]:
final['window'] = final['window'].fillna('Во двор' or 'На улицу' or 'На улицу и двор')
final.rename(columns = {'Окна_Во двор' : 'window_backyard', 'Окна_На улицу' : 'window_street', 'Окна_На улицу и двор' : 'window_backyard_street'}, inplace = True)
final = pd.concat([final, pd.get_dummies(final['window'], prefix='Окна')], axis=1)
# df_2.drop('Окна', axis=1, inplace=True)
final.rename(columns = {'Окна_Во двор' : 'window_backyard', 'Окна_На улицу' : 'window_street', 'Окна_На улицу и двор' : 'window_backyard_street'}, inplace = True)

In [1380]:
final.columns

Index(['Количество комнат', 'type_of_room', 'Метро', 'Адрес', 'Площадь, м2',
       'Дом', 'parking', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'window',
       'Можно с детьми/животными', 'Дополнительно', 'Высота потолков, м',
       'Лифт', 'City', 'Street', 'Number', 'city_moscow',
       'city_saint_petersburg', 'city_krasnodar_region', 'adres_t',
       'garbage_chute', 'building_level', 'building_сount_levels',
       'elevator_passenger_cargo', 'elevator_passenger', 'elevator_cargo',
       'repair_designer', 'repair_renovation', 'repair_redecorating',
       'repair_not_specified', 'repair_without_repair', 'window_backyard',
       'window_street', 'window_backyard_street'],
      dtype='object')

In [1381]:
final = final.drop(['Количество комнат', 'Метро', 'Адрес', 'Площадь, м2', 'Дом', 'Цена', 'Описание', 'Ремонт', 'Балкон', 'window', 'Можно с детьми/животными', 'Дополнительно'], axis=1)

In [1382]:
display_missing(final)

type_of_room Количество пропущенных значений: 0
parking Количество пропущенных значений: 0
Высота потолков, м Количество пропущенных значений: 0
Лифт Количество пропущенных значений: 0
City Количество пропущенных значений: 0
Street Количество пропущенных значений: 0
Number Количество пропущенных значений: 4
city_moscow Количество пропущенных значений: 0
city_saint_petersburg Количество пропущенных значений: 0
city_krasnodar_region Количество пропущенных значений: 0
adres_t Количество пропущенных значений: 0
garbage_chute Количество пропущенных значений: 0
building_level Количество пропущенных значений: 0
building_сount_levels Количество пропущенных значений: 0
elevator_passenger_cargo Количество пропущенных значений: 0
elevator_passenger Количество пропущенных значений: 0
elevator_cargo Количество пропущенных значений: 0
repair_designer Количество пропущенных значений: 0
repair_renovation Количество пропущенных значений: 0
repair_redecorating Количество пропущенных значений: 0
repair_n

In [1383]:
final.dtypes

type_of_room                 object
parking                       int64
Высота потолков, м          float64
Лифт                         object
City                         object
Street                       object
Number                       object
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
adres_t                      object
garbage_chute                 int64
building_level                int64
building_сount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               uint8
window_street                 uint8
window_backyard_street        uint8
dtype: object

In [1384]:
final.rename(columns = {'Высота потолков, м' : 'ceiling_height'}, inplace = True)

In [1385]:
final.dtypes

type_of_room                 object
parking                       int64
ceiling_height              float64
Лифт                         object
City                         object
Street                       object
Number                       object
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
adres_t                      object
garbage_chute                 int64
building_level                int64
building_сount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               uint8
window_street                 uint8
window_backyard_street        uint8
dtype: object

In [1386]:
final = final.drop(['Лифт', 'City', 'Street', 'Number', 'adres_t'], axis=1)

In [1387]:
final.head(10)

,type_of_room,parking,ceiling_height,city_moscow,city_saint_petersburg,city_krasnodar_region,garbage_chute,building_level,building_сount_levels,elevator_passenger_cargo,elevator_passenger,elevator_cargo,repair_designer,repair_renovation,repair_redecorating,repair_not_specified,repair_without_repair,window_backyard,window_street,window_backyard_street
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
0,Квартира,0,3.0,1.0,0.0,0.0,1,5,16,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0
1,Квартира,0,3.5,1.0,0.0,0.0,0,5,16,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0,1
2,Квартира,0,3.2,1.0,0.0,0.0,1,5,16,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,1
3,Квартира,0,3.2,1.0,0.0,0.0,1,5,6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,1
4,Квартира,0,3.9,1.0,0.0,0.0,1,12,26,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,1
5,Квартира,0,3.5,1.0,0.0,0.0,1,3,7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0
6,Квартира,1,3.2,1.0,0.0,0.0,0,4,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,1
7,Квартира,0,3.2,1.0,0.0,0.0,0,2,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0
8,Квартира,0,3.0,1.0,0.0,0.0,0,5,10,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,0


In [1388]:
final = final.drop(['type_of_room'], axis=1)

In [1389]:
final.columns

Index(['parking', 'ceiling_height', 'city_moscow', 'city_saint_petersburg',
       'city_krasnodar_region', 'garbage_chute', 'building_level',
       'building_сount_levels', 'elevator_passenger_cargo',
       'elevator_passenger', 'elevator_cargo', 'repair_designer',
       'repair_renovation', 'repair_redecorating', 'repair_not_specified',
       'repair_without_repair', 'window_backyard', 'window_street',
       'window_backyard_street'],
      dtype='object')

In [1390]:
final.to_csv('data_clean_cian.csv', sep='\t')

In [1391]:
display_missing(final)

parking Количество пропущенных значений: 0
ceiling_height Количество пропущенных значений: 0
city_moscow Количество пропущенных значений: 0
city_saint_petersburg Количество пропущенных значений: 0
city_krasnodar_region Количество пропущенных значений: 0
garbage_chute Количество пропущенных значений: 0
building_level Количество пропущенных значений: 0
building_сount_levels Количество пропущенных значений: 0
elevator_passenger_cargo Количество пропущенных значений: 0
elevator_passenger Количество пропущенных значений: 0
elevator_cargo Количество пропущенных значений: 0
repair_designer Количество пропущенных значений: 0
repair_renovation Количество пропущенных значений: 0
repair_redecorating Количество пропущенных значений: 0
repair_not_specified Количество пропущенных значений: 0
repair_without_repair Количество пропущенных значений: 0
window_backyard Количество пропущенных значений: 0
window_street Количество пропущенных значений: 0
window_backyard_street Количество пропущенных значений

In [1393]:
final.dtypes

parking                       int64
ceiling_height              float64
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
garbage_chute                 int64
building_level                int64
building_сount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               uint8
window_street                 uint8
window_backyard_street        uint8
dtype: object

In [3]:
c = pd.read_csv('data_clean_cian_2.csv')

In [4]:
def display_missing(df):    
    for col in df.columns.tolist():          
        print('{} Количество пропущенных значений: {}'.format(col, df[col].isnull().sum()))
    print('\n')

In [9]:
display_missing(c)

Unnamed: 0 Количество пропущенных значений: 0
parking Количество пропущенных значений: 0
ceiling_height Количество пропущенных значений: 0
city_moscow Количество пропущенных значений: 0
city_saint_petersburg Количество пропущенных значений: 0
city_krasnodar_region Количество пропущенных значений: 0
garbage_chute Количество пропущенных значений: 0
building_level Количество пропущенных значений: 0
building_сount_levels Количество пропущенных значений: 0
elevator_passenger_cargo Количество пропущенных значений: 0
elevator_passenger Количество пропущенных значений: 0
elevator_cargo Количество пропущенных значений: 0
repair_designer Количество пропущенных значений: 0
repair_renovation Количество пропущенных значений: 0
repair_redecorating Количество пропущенных значений: 0
repair_not_specified Количество пропущенных значений: 0
repair_without_repair Количество пропущенных значений: 0
window_backyard Количество пропущенных значений: 0
window_street Количество пропущенных значений: 0
window_b

In [10]:
c.dtypes

Unnamed: 0                    int64
parking                       int64
ceiling_height              float64
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
garbage_chute                 int64
building_level                int64
building_сount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               int64
window_street                 int64
window_backyard_street        int64
kolichestvo_komnat            int64
smezhniye_komnatyi            int64
izolirovaniye_komnatyi        int64
metro                         int64
ploschad_m2                 float64
balcon                        int64
lodjia                        int64
vanna                       

In [11]:
c['tsena2']

0        500000
1        500000
2        500000
3        400000
4        225000
          ...  
23363     42000
23364     45000
23365     50000
23366     55000
23367     57000
Name: tsena2, Length: 23368, dtype: object

In [21]:
p = pd.read_csv('data.csv', encoding='latin-1')

In [22]:
p.dtypes

Unnamed: 0                    int64
parking                       int64
ceiling_height              float64
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
garbage_chute                 int64
building_level                int64
building_ñount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               int64
window_street                 int64
window_backyard_street        int64
kolichestvo_komnat            int64
smezhniye_komnatyi            int64
izolirovaniye_komnatyi        int64
metro                         int64
ploschad_m2                 float64
balcon                        int64
lodjia                        int64
vanna                       

In [25]:
p['tsena2'] = p['tsena2'].str.replace('?', '')
p['tsena2'] = p['tsena2'].astype(float)

/tmp/ipykernel_4404/217945485.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  p['tsena2'] = p['tsena2'].str.replace('?', '')


In [27]:
p.dtypes

Unnamed: 0                    int64
parking                       int64
ceiling_height              float64
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
garbage_chute                 int64
building_level                int64
building_ñount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               int64
window_street                 int64
window_backyard_street        int64
kolichestvo_komnat            int64
smezhniye_komnatyi            int64
izolirovaniye_komnatyi        int64
metro                         int64
ploschad_m2                 float64
balcon                        int64
lodjia                        int64
vanna                       

In [29]:
p['zalog'] = p['zalog'].str.replace('?', '')
p['zalog'] = p['zalog'].astype(float)

/tmp/ipykernel_4404/4060282694.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  p['zalog'] = p['zalog'].str.replace('?', '')


In [33]:
p['predoplata'] = p['predoplata'].str.replace('?', '').str.replace('#', '').str.replace('!', '')
p['predoplata'] = p['predoplata'].astype(float)

/tmp/ipykernel_4404/1606049908.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  p['predoplata'] = p['predoplata'].str.replace('?', '').str.replace('#', '').str.replace('!', '')


ValueError: could not convert string to float: ''

In [32]:
p.dtypes

Unnamed: 0                    int64
parking                       int64
ceiling_height              float64
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
garbage_chute                 int64
building_level                int64
building_ñount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               int64
window_street                 int64
window_backyard_street        int64
kolichestvo_komnat            int64
smezhniye_komnatyi            int64
izolirovaniye_komnatyi        int64
metro                         int64
ploschad_m2                 float64
balcon                        int64
lodjia                        int64
vanna                       

In [34]:
p = p.drop(['predoplata'], axis=1)

In [35]:
p.dtypes

Unnamed: 0                    int64
parking                       int64
ceiling_height              float64
city_moscow                 float64
city_saint_petersburg       float64
city_krasnodar_region       float64
garbage_chute                 int64
building_level                int64
building_ñount_levels         int64
elevator_passenger_cargo    float64
elevator_passenger          float64
elevator_cargo              float64
repair_designer             float64
repair_renovation           float64
repair_redecorating         float64
repair_not_specified        float64
repair_without_repair       float64
window_backyard               int64
window_street                 int64
window_backyard_street        int64
kolichestvo_komnat            int64
smezhniye_komnatyi            int64
izolirovaniye_komnatyi        int64
metro                         int64
ploschad_m2                 float64
balcon                        int64
lodjia                        int64
vanna                       

In [36]:
p.to_csv('data_clean.csv', sep='\t')

In [37]:
p.to_csv('data_clean_cian_new.csv', sep='\t')